
## Momentum Score Implementation

Link - https://www.msci.com/eqb/methodology/meth_docs/MSCI_Momentum_Indexes_Methodology_Aug2021.pdf

In [215]:
# Important libraries used

import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import time
import pickle

# To diable yahoo finance logging processing messages
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True
logger.propagate = False

### Universe
- Nifty 500
- S&P 500

In [216]:
# Uploading list of tickers for S&P500 and Nifty500 - Universe

nf500 = pd.read_csv("ind_nifty500list.csv")
sp500 = pd.read_csv("sp500_companies.csv")

In [217]:
# Checking columns
nf500.columns, sp500.columns

(Index(['Company Name', 'Industry', 'Symbol', 'Series', 'ISIN Code'], dtype='object'),
 Index(['Exchange', 'Symbol', 'Shortname', 'Longname', 'Sector', 'Industry',
        'Currentprice', 'Marketcap', 'Ebitda', 'Revenuegrowth', 'City', 'State',
        'Country', 'Fulltimeemployees', 'Longbusinesssummary', 'Weight'],
       dtype='object'))

In [218]:
# Check that market cap is sorted for S&P 500
print(sum(sp500.Marketcap - sp500.Marketcap.sort_values()))
print(sp500.shape)

0
(503, 16)


In [219]:
# Seperating tickers for use

nf_tickers = list(nf500.Symbol)
sp_tickers = list(sp500.Symbol)

sp_ohlcv = {}
nf_ohlcv = {}

### Download Data

Based on the MSCI paper we need data for a time period of 1 week i.e. 5 days.

Using a lookback of 10 years.

### BE AWARE OF DOWNLOAD CALL CELLS THEY TAKE TIME TO FETCH
#### MARKED WITH "CAUTION"

In [220]:
# Download data with time delay

def download_data(ohlcv, tickers, start, end, period = "5d", region="", left=0, right=-1):
    idx = 0
    for ticker in tickers[left:right]:
        ohlcv[ticker] = yf.download(ticker +region, start, end, period = period).loc[:, ["Adj Close"]]
        print(left+idx)
        idx += 1
        # time.sleep(2) # Added sleep time not to overload the server

In [221]:
# Nifty 500 Data

start = dt.datetime.today() - dt.timedelta(3650)
end = dt.datetime.today()



# Use the pickle files provided with the code instead to load data Uncomment the code below
###############################################

with open("nifty_500.pickle" , 'rb') as f:
    nf_ohlcv = pickle.load(f)

###############################################


# CAUTION
# This step will take 25 mins approx Run separately or together based on time

# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 0, right=100)

# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 100, right = 200)

# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 200, right = 300)

# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 300, right = 400)

# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 400, right = -1)



In [223]:
# Remove empty dataframe and save the data to pickle file

empty = []
for ticker in nf_tickers:
    if ticker in nf_ohlcv:
        if nf_ohlcv[ticker].shape[0]==0:
            empty.append(ticker)
    else:
        empty.append(ticker)
        
print(empty)
for ticker in empty:
    nf_ohlcv.pop(ticker, None)
    nf_tickers.remove(ticker)

print(len(nf_ohlcv.keys()), len(nf_tickers))

# Saving data to pickle file to avoid re-download
# with open('nifty_500.pickle', 'wb') as f:
#     pickle.dump(nf_ohlcv, f)

['DUMMYRAYMD', 'DUMMYSANOF']
501 501


In [224]:
# S&P 500 Data


# Use the pickle files provided with the code instead to load data Uncomment the code below
############################################

with open("sp_500.pickle" , 'rb') as f:
    sp_ohlcv = pickle.load(f)

############################################


# CAUTION
# This step will take 15 mins approx Run separately or together based on time - Also uncomment the save code in next code block

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=0, right=100)

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=100, right=200)

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=200, right=300)

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=300, right=400)

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=400, right=-1)

In [225]:
# Remove empty dataframe and save the data to pickle file

empty = []
for ticker in sp_tickers:
    if ticker in sp_ohlcv:
        if sp_ohlcv[ticker].shape[0]==0:
            empty.append(ticker)
    else:
        empty.append(ticker)
print(empty)
for ticker in empty:
    sp_ohlcv.pop(ticker, None)
    sp_tickers.remove(ticker)

print(len(sp_ohlcv.keys()), len(sp_tickers))


# Saving data to pickle file to avoid re-download
# with open('sp_500.pickle', 'wb') as f:
#     pickle.dump(sp_ohlcv, f)

['ETSY']
502 502


### Determination of Momentum Score

In [226]:
# Monthly Price Momentum
def price_momentum(data, period, tickers, local_risk_free_rate):
    for ticker in tickers:
        data[ticker][str(period)+"_mon_price_mom"] = (data[ticker]["Adj Close"].shift(int(252/52))/data[ticker]["Adj Close"].shift(int((period+1)*252/52))-1) - local_risk_free_rate


In [228]:
# 6 and 12-month price momentum # local risk free rate taken from 5 year treasury bond yield - INDIA and USA - Recommendation from Business Valuation by Damodaran

price_momentum(nf_ohlcv, period=6, tickers=nf_tickers, local_risk_free_rate=.06889)
price_momentum(nf_ohlcv, period=12, tickers=nf_tickers, local_risk_free_rate=.06889)

# 6 and 12-month price momentum # local risk free rate taken from 5 year treasury bond yield - INDIA and USA

price_momentum(sp_ohlcv, period=12, tickers=sp_tickers, local_risk_free_rate=.036)
price_momentum(sp_ohlcv, period=6, tickers=sp_tickers, local_risk_free_rate=.036)

In [229]:
# checking calculation for one stock

nf_ohlcv["360ONE"]

,Adj Close,6_mon_price_mom,12_mon_price_mom
Date,,,
2019-09-19,262.504547,NaN,NaN
2019-09-20,275.624512,NaN,NaN
2019-09-23,289.405792,NaN,NaN
2019-09-24,303.868958,NaN,NaN
2019-09-25,290.283875,NaN,NaN
...,...,...,...
2024-08-16,1069.599976,-0.011955,0.303417
2024-08-19,1098.050049,-0.002230,0.288499
2024-08-20,1098.750000,-0.002965,0.220411


### Risk-Adjusted Momentum Value

In [230]:
# First we need to calculate volatility for each for 7 day price and 3 year period
def volatility(data, tickers, window_size):
    for ticker in tickers:
        data[ticker]["7day_3year_sigma"] = data[ticker]["Adj Close"].pct_change().rolling(window_size).std()*(52)

In [234]:
# Calculating for both Nifty and S&P
# Window size is for 3 years

volatility(nf_ohlcv, tickers=nf_tickers, window_size=3*52)
volatility(sp_ohlcv, tickers=sp_tickers, window_size=3*52)

In [235]:
# Risk adjusted momentul for 6 and 12 months
def risk_adjusted_momentum(data, tickers, period):
    for ticker in tickers:
        data[ticker][str(period)+"_mon_risk_adj_mom"] = data[ticker][str(period)+"_mon_price_mom"]/data[ticker]["7day_3year_sigma"]

In [236]:
# 6 and 12 month adjustment for NIFTY500
risk_adjusted_momentum(nf_ohlcv, tickers=nf_tickers, period=6)
risk_adjusted_momentum(nf_ohlcv, tickers=nf_tickers, period=12)

# 6 and 12 month adjustment for S&P500
risk_adjusted_momentum(sp_ohlcv, tickers=sp_tickers, period=6)
risk_adjusted_momentum(sp_ohlcv, tickers=sp_tickers, period=12)

In [237]:
# Checking calculations so far

nf_ohlcv["360ONE"]

,Adj Close,6_mon_price_mom,12_mon_price_mom,7day_3year_sigma,6_mon_risk_adj_mom,12_mon_risk_adj_mom
Date,,,,,,
2019-09-19,262.504547,NaN,NaN,NaN,NaN,NaN
2019-09-20,275.624512,NaN,NaN,NaN,NaN,NaN
2019-09-23,289.405792,NaN,NaN,NaN,NaN,NaN
2019-09-24,303.868958,NaN,NaN,NaN,NaN,NaN
2019-09-25,290.283875,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-08-16,1069.599976,-0.011955,0.303417,1.296532,-0.009221,0.234022
2024-08-19,1098.050049,-0.002230,0.288499,1.293352,-0.001724,0.223063
2024-08-20,1098.750000,-0.002965,0.220411,1.278052,-0.002320,0.172459


### Calculating Momentum Score

In [238]:
# Calculating Z
# Doubt on window size not clear from the doc
def zscore(data, period, window, tickers):
    for ticker in tickers:
        data[ticker][str(period)+"_mon_mom_zscore"] = (data[ticker][str(period)+"_mon_risk_adj_mom"] - data[ticker][str(period)+"_mon_risk_adj_mom"].rolling(window=window).mean()) / data[ticker][str(period)+"_mon_risk_adj_mom"].rolling(window=window).std()

In [239]:
# Zscores for all the four cases
# 6 and 12 month Zscores
zscore(nf_ohlcv, period = 6, window=3*52, tickers=nf_tickers)
zscore(nf_ohlcv, period = 12, window=3*52, tickers=nf_tickers)

# 6 and 12 month Zscores
zscore(sp_ohlcv, period = 6, window=3*52, tickers=sp_tickers)
zscore(sp_ohlcv, period = 12, window=3*52, tickers=sp_tickers)

In [240]:
# Calculation check
nf_ohlcv["360ONE"]

,Adj Close,6_mon_price_mom,12_mon_price_mom,7day_3year_sigma,6_mon_risk_adj_mom,12_mon_risk_adj_mom,6_mon_mom_zscore,12_mon_mom_zscore
Date,,,,,,,,
2019-09-19,262.504547,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-20,275.624512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-23,289.405792,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-24,303.868958,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-25,290.283875,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-08-16,1069.599976,-0.011955,0.303417,1.296532,-0.009221,0.234022,-0.485124,1.123830
2024-08-19,1098.050049,-0.002230,0.288499,1.293352,-0.001724,0.223063,-0.393658,1.003955
2024-08-20,1098.750000,-0.002965,0.220411,1.278052,-0.002320,0.172459,-0.392351,0.486178


In [241]:
# Calulating Single Momentum Combined Score (C)
def cCal(data, tickers):
    for ticker in tickers:
        data[ticker]["C_Calc"] = 0.5 * data[ticker]["6_mon_mom_zscore"] + 0.5 * data[ticker]["12_mon_mom_zscore"]
        

In [242]:
# Single Momentum Combined Score for all the four cases
# 6 and 12 month Zscores
cCal(nf_ohlcv, tickers=nf_tickers)
cCal(nf_ohlcv, tickers=nf_tickers)

# 6 and 12 month Zscores
cCal(sp_ohlcv, tickers=sp_tickers)
cCal(sp_ohlcv, tickers=sp_tickers)

### Calculating Single Momentum Standardized Z-score

In [289]:
def sms_zscore(data, window, tickers):
    for ticker in tickers:
        data[ticker][str(window)+"_sms_zscore"] = (data[ticker]["C_Calc"] - data[ticker]["C_Calc"].rolling(window=window).mean()) / data[ticker]["C_Calc"].rolling(window=window).std()
        
        # Now we winsorize it
        data[ticker][str(window)+"_sms_zscore"] = data[ticker][str(window)+"_sms_zscore"].apply(lambda x: max(min(x, 3), -3))

In [290]:
# Single Momentum Combined Score for all the four cases
# 6 and 12 month Zscores
sms_zscore(nf_ohlcv, window=3*52, tickers=nf_tickers)

# 6 and 12 month Zscores
sms_zscore(sp_ohlcv, window=3*52, tickers=sp_tickers)

### Picking Top 10 out of 500

In [291]:
# Take all the present day values of C scores for all the 500 stocks and their tickers and save in another data frame

latest_C_nf = pd.DataFrame(columns=["ticker", "Winsorized_Score"])
latest_C_sp = pd.DataFrame(columns=["ticker", "Winsorized_Score"])

def pickScores(data, output, tickers):
    for idx, ticker in enumerate(tickers):
        output.at[idx,"ticker"] = ticker 
        output.at[idx,"Winsorized_Score"] = data[ticker]["156_sms_zscore"][-1]

pickScores(nf_ohlcv, latest_C_nf, nf_tickers)
pickScores(sp_ohlcv, latest_C_sp, sp_tickers)


### Results

In [298]:
# Dropping winsorized score for those which the data was insufficient to calculate
nifty_na = latest_C_nf[latest_C_nf["Winsorized_Score"].isna()]

nifty_without_na = latest_C_nf.dropna()

top10_nf = nifty_without_na.sort_values(by="Winsorized_Score", ascending=False).head(10).merge(nf500, how = "left", left_on="ticker", right_on="Symbol")

print("Top 10 picks based on {} calculations".format(dt.datetime.today().date()))
top10_nf[["Company Name", "Symbol", "Industry", "Winsorized_Score"]]

Top 10 picks based on 2024-08-23 calculations


,Company Name,Symbol,Industry,Winsorized_Score
0,Colgate Palmolive (India) Ltd.,COLPAL,Fast Moving Consumer Goods,2.95044
1,Multi Commodity Exchange of India Ltd.,MCX,Financial Services,2.73205
2,CCL Products (I) Ltd.,CCL,Fast Moving Consumer Goods,2.674321
3,Anupam Rasayan India Ltd.,ANURAS,Chemicals,2.608056
4,Central Depository Services (India) Ltd.,CDSL,Financial Services,2.603077
5,Ajanta Pharmaceuticals Ltd.,AJANTPHARM,Healthcare,2.555692
6,PCBL Ltd.,PCBL,Chemicals,2.349055
7,Suzlon Energy Ltd.,SUZLON,Capital Goods,2.126143
8,BSE Ltd.,BSE,Financial Services,1.990239
9,Berger Paints India Ltd.,BERGEPAINT,Consumer Durables,1.961197


In [297]:
# Dropping winsorized score for those which the data was insufficient to calculate
sp_na = latest_C_sp[latest_C_sp["Winsorized_Score"].isna()]

sp_without_na = latest_C_sp.dropna()

top10_sp = sp_without_na.sort_values(by="Winsorized_Score", ascending=False).head(10).merge(sp500, how = "left", left_on="ticker", right_on="Symbol")

print("Top 10 picks based on {} calculations".format(dt.datetime.today().date()))
top10_sp[["Shortname", "Symbol", "Industry", "Winsorized_Score", "Marketcap"]]

Top 10 picks based on 2024-08-23 calculations


,Shortname,Symbol,Industry,Winsorized_Score,Marketcap
0,"Cardinal Health, Inc.",CAH,Medical Distribution,3.0,26737604608
1,"Nike, Inc.",NKE,Footwear & Accessories,3.0,125946241024
2,Starbucks Corporation,SBUX,Restaurants,3.0,105138290688
3,"Cisco Systems, Inc.",CSCO,Communication Equipment,3.0,203253465088
4,"Assurant, Inc.",AIZ,Insurance - Specialty,3.0,9831922688
5,"Becton, Dickinson and Company",BDX,Medical Instruments & Supplies,2.849662,67881512960
6,McDonald's Corporation,MCD,Restaurants,2.652662,207821422592
7,Kellanova,K,Packaged Foods,2.598798,27718688768
8,Coca-Cola Company (The),KO,Beverages - Non-Alcoholic,2.574359,299837652992
9,Atmos Energy Corporation,ATO,Utilities - Regulated Gas,2.509706,20116643840
